# Lab 3.4.4: LangGraph Workflow with Human-in-the-Loop

**Module:** 3.4 - AI Agents & Agentic Systems  
**Time:** 3 hours  
**Difficulty:** ⭐⭐⭐⭐ (Advanced)

---

## 🎯 Learning Objectives

By the end of this notebook, you will:
- [ ] Understand graph-based agent orchestration
- [ ] Build stateful workflows with LangGraph
- [ ] Implement human-in-the-loop approval checkpoints
- [ ] Handle complex branching and conditional logic
- [ ] Create persistent, resumable agent workflows

---

## 📚 Prerequisites

- Completed: Tasks 13.1-13.3
- Knowledge of: State machines, graph concepts

---

## 🌍 Real-World Context

**Why do we need graph-based workflows?**

Simple chains work for linear tasks, but real-world processes are complex:
- 🔄 **Loops**: "Keep refining until the user approves"
- 🔀 **Branches**: "If high-risk, get manager approval"
- ⏸️ **Pauses**: "Wait for human review before proceeding"
- 💾 **Memory**: "Remember what happened across sessions"

**Real examples:**
- ✍️ **Content Creation**: Draft → Review → Revise → Approve → Publish
- 🏦 **Loan Processing**: Application → Risk Assessment → Human Review → Decision
- 🔧 **IT Tickets**: Classify → Route → Escalate if needed → Resolve → Verify

---

## 🧒 ELI5: What is LangGraph?

> **Imagine a board game where the AI is a player...** 🎲
>
> In most AI applications, it's like playing a game where you can only move forward:
> - Start → Do thing 1 → Do thing 2 → Done!
>
> But real games have more options:
> - You might go back ("Oops, try again!")
> - You might branch ("If you roll 6, go here")
> - You might wait ("Player 2's turn first")
>
> **LangGraph is like the game board!**
> - Each square is a "node" (something that happens)
> - Lines between squares are "edges" (what happens next)
> - The game piece remembers where it's been ("state")
>
> ```
>        ┌──────────┐
>        │  START   │
>        └────┬─────┘
>             │
>             ▼
>        ┌──────────┐
>        │  DRAFT   │
>        └────┬─────┘
>             │
>             ▼
>        ┌──────────┐     ┌──────────┐
>        │  REVIEW  │────►│  REVISE  │
>        └────┬─────┘     └────┬─────┘
>             │                 │
>        approved           loops back
>             │                 │
>             ▼                 │
>        ┌──────────┐          │
>        │ PUBLISH  │◄─────────┘
>        └──────────┘
>   ```

---

## Part 1: Environment Setup

In [ ]:
# Install LangGraph (run once)
# Pinned version for reproducibility - update as needed
# !pip install langgraph>=0.1.0

In [ ]:
# Standard imports
import os
import sys
from pathlib import Path
from typing import TypedDict, Annotated, List, Optional, Literal
import json
import time
from datetime import datetime

# LangGraph imports with version compatibility
from langgraph.graph import StateGraph, END

# Handle different LangGraph versions for MemorySaver
try:
    from langgraph.checkpoint.memory import MemorySaver  # langgraph >= 0.1.0
except ImportError:
    try:
        from langgraph.checkpoint import MemorySaver  # langgraph < 0.1.0
    except ImportError:
        print("⚠️ MemorySaver not available. Install langgraph: pip install langgraph>=0.1.0")
        MemorySaver = None

# LangChain imports
from langchain_community.llms import Ollama
from langchain.prompts import PromptTemplate

print("✅ Imports successful!")

In [ ]:
# Model Configuration - Change this if using a different model
LLM_MODEL = "llama3.1:8b"  # Options: "llama3.1:8b", "llama3.1:70b", "mistral:7b"

# Initialize LLM with timeout for long operations
llm = Ollama(
    model=LLM_MODEL,
    temperature=0.7,
    request_timeout=120.0,  # 2 minute timeout
    base_url="http://localhost:11434"
)

print(f"✅ LLM initialized: {LLM_MODEL}")

---

## Part 2: Understanding State

In LangGraph, **state** is the data that flows through your graph and gets updated at each step.

In [ ]:
# Define the state for our content creation workflow
class ContentState(TypedDict):
    """State for the content creation workflow."""
    # Input
    topic: str
    content_type: str  # "blog", "tweet", "email"
    
    # Generated content
    draft: str
    revised_draft: str
    
    # Review state
    feedback: str
    approved: bool
    revision_count: int
    max_revisions: int
    
    # Metadata
    messages: List[str]  # Log of what happened

print("ContentState defined!")
print("Fields:", list(ContentState.__annotations__.keys()))

---

## Part 3: Building the Graph Nodes

Each node is a function that takes state, does something, and returns updated state.

In [ ]:
# Node 1: Generate initial draft
def generate_draft(state: ContentState) -> ContentState:
    """Generate the initial content draft."""
    print("📝 Generating draft...")
    
    prompt = f"""Write a {state['content_type']} about: {state['topic']}
    
Keep it concise, engaging, and professional.
Only output the content itself, no preamble."""
    
    draft = llm.invoke(prompt)
    
    state['draft'] = draft
    state['messages'].append(f"Draft generated at {datetime.now().strftime('%H:%M:%S')}")
    
    return state

print("Node 'generate_draft' defined!")

In [ ]:
# Node 2: Review the draft (simulated or real human review)
def review_draft(state: ContentState) -> ContentState:
    """Review the draft and provide feedback."""
    print("🔍 Reviewing draft...")
    
    # In production, this could wait for human input!
    # Here we simulate AI review
    
    current_draft = state.get('revised_draft') or state['draft']
    
    review_prompt = f"""Review this {state['content_type']} and provide brief feedback.
If it's good, say "APPROVED: [brief praise]".
If it needs work, say "REVISE: [specific feedback]".

Content to review:
{current_draft}"""
    
    feedback = llm.invoke(review_prompt)
    
    state['feedback'] = feedback
    state['approved'] = "APPROVED" in feedback.upper()
    state['messages'].append(f"Review complete: {'Approved' if state['approved'] else 'Needs revision'}")
    
    return state

print("Node 'review_draft' defined!")

In [ ]:
# Node 3: Revise the draft based on feedback
def revise_draft(state: ContentState) -> ContentState:
    """Revise the draft based on feedback."""
    print(f"✏️ Revising draft (attempt {state['revision_count'] + 1})...")
    
    current_draft = state.get('revised_draft') or state['draft']
    
    revise_prompt = f"""Revise this {state['content_type']} based on the feedback.

Original content:
{current_draft}

Feedback:
{state['feedback']}

Provide the revised content only, no preamble."""
    
    revised = llm.invoke(revise_prompt)
    
    state['revised_draft'] = revised
    state['revision_count'] += 1
    state['messages'].append(f"Revision {state['revision_count']} complete")
    
    return state

print("Node 'revise_draft' defined!")

In [ ]:
# Node 4: Publish (finalize)
def publish_content(state: ContentState) -> ContentState:
    """Publish the final approved content."""
    print("🎉 Publishing content!")
    
    final_content = state.get('revised_draft') or state['draft']
    state['messages'].append(f"Published at {datetime.now().strftime('%H:%M:%S')}")
    
    print("\n" + "="*60)
    print("FINAL PUBLISHED CONTENT")
    print("="*60)
    print(final_content)
    print("="*60)
    
    return state

print("Node 'publish_content' defined!")

---

## Part 4: Defining Edges (Routing Logic)

Edges define how the graph flows from one node to another.

In [ ]:
# Conditional edge: After review, decide what to do next
def should_continue(state: ContentState) -> Literal["revise", "publish", "end"]:
    """Decide whether to revise, publish, or end."""
    
    if state['approved']:
        print("  → Content approved! Moving to publish...")
        return "publish"
    
    if state['revision_count'] >= state['max_revisions']:
        print(f"  → Max revisions ({state['max_revisions']}) reached. Publishing anyway...")
        return "publish"
    
    print("  → Needs revision...")
    return "revise"

print("Routing function 'should_continue' defined!")

---

## Part 5: Building the Graph

In [ ]:
# Create the graph
workflow = StateGraph(ContentState)

# Add nodes
workflow.add_node("generate", generate_draft)
workflow.add_node("review", review_draft)
workflow.add_node("revise", revise_draft)
workflow.add_node("publish", publish_content)

# Add edges
workflow.set_entry_point("generate")  # Start here
workflow.add_edge("generate", "review")  # Always review after generating

# Conditional edge after review
workflow.add_conditional_edges(
    "review",
    should_continue,
    {
        "revise": "revise",
        "publish": "publish",
    }
)

# After revision, go back to review
workflow.add_edge("revise", "review")

# Publish ends the workflow
workflow.add_edge("publish", END)

print("Graph structure defined!")

In [ ]:
# Compile the graph
app = workflow.compile()

print("Graph compiled successfully!")

In [ ]:
# Visualize the graph (if graphviz/mermaid is available)
try:
    from IPython.display import Image, display
    display(Image(app.get_graph().draw_mermaid_png()))
    print("✅ Graph visualization rendered!")
except ImportError as e:
    print(f"⚠️ Visualization requires additional packages: {e}")
    print("   Install with: pip install grandalf")
    print("\nGraph structure (text representation):")
    print("""
    ┌──────────┐
    │ generate │
    └────┬─────┘
         │
         ▼
    ┌──────────┐
    │  review  │◄──────────┐
    └────┬─────┘           │
         │                 │
    (approved?)        ┌───────┐
     │      │          │revise │
     │      └─────────►└───────┘
     │
     ▼
    ┌──────────┐
    │ publish  │
    └────┬─────┘
         │
         ▼
       [END]
    """)
except Exception as e:
    print(f"⚠️ Visualization not available: {e}")
    print("Graph flows: generate → review → (approve?) → publish → END")
    print("                         ↑         ↓")
    print("                         ← revise ←")

---

## Part 6: Running the Workflow

In [ ]:
# Initialize the state
initial_state = {
    "topic": "The benefits of AI agents in software development",
    "content_type": "tweet",
    "draft": "",
    "revised_draft": "",
    "feedback": "",
    "approved": False,
    "revision_count": 0,
    "max_revisions": 2,
    "messages": []
}

print("Initial state created!")
print(f"Topic: {initial_state['topic']}")
print(f"Content type: {initial_state['content_type']}")

In [ ]:
# Run the workflow
print("\n" + "="*60)
print("RUNNING CONTENT CREATION WORKFLOW")
print("="*60 + "\n")

start_time = time.time()

# Stream the execution to see each step
for step in app.stream(initial_state):
    # step is a dict with node_name: output_state
    for node_name, output in step.items():
        print(f"\n--- Completed node: {node_name} ---")
        
elapsed = time.time() - start_time

print(f"\nWorkflow completed in {elapsed:.1f} seconds!")

In [ ]:
# Get the final result
final_result = app.invoke(initial_state)

print("\n" + "="*60)
print("WORKFLOW SUMMARY")
print("="*60)
print(f"Revisions made: {final_result['revision_count']}")
print(f"Final approval: {final_result['approved']}")
print(f"\nExecution log:")
for msg in final_result['messages']:
    print(f"  • {msg}")

---

## Part 7: Human-in-the-Loop

Now let's add the ability for humans to approve or provide feedback!

In [ ]:
# Create a version with interrupt points for human review

# Modified review node that can accept human input
def human_review(state: ContentState) -> ContentState:
    """Request human review of the draft."""
    current_draft = state.get('revised_draft') or state['draft']
    
    print("\n" + "="*60)
    print("🧑 HUMAN REVIEW REQUESTED")
    print("="*60)
    print(f"\nContent to review ({state['content_type']}):")
    print("-"*40)
    print(current_draft)
    print("-"*40)
    
    # In a real application, this would wait for actual human input
    # For demo, we'll simulate it
    print("\nSimulating human review...")
    
    # Simulate human decision (in production: get real input)
    if state['revision_count'] >= 1:  # Approve after 1 revision
        feedback = "APPROVED: Good job!"
    else:
        feedback = "REVISE: Please make it more engaging and add an emoji."
    
    print(f"Human feedback: {feedback}")
    
    state['feedback'] = feedback
    state['approved'] = "APPROVED" in feedback.upper()
    state['messages'].append(f"Human review: {'Approved' if state['approved'] else 'Needs revision'}")
    
    return state

In [ ]:
# Create a new graph with human review
hitl_workflow = StateGraph(ContentState)

# Add nodes
hitl_workflow.add_node("generate", generate_draft)
hitl_workflow.add_node("human_review", human_review)  # Human in the loop!
hitl_workflow.add_node("revise", revise_draft)
hitl_workflow.add_node("publish", publish_content)

# Add edges
hitl_workflow.set_entry_point("generate")
hitl_workflow.add_edge("generate", "human_review")

hitl_workflow.add_conditional_edges(
    "human_review",
    should_continue,
    {
        "revise": "revise",
        "publish": "publish",
    }
)

hitl_workflow.add_edge("revise", "human_review")
hitl_workflow.add_edge("publish", END)

# Compile with memory for persistence
memory = MemorySaver()
hitl_app = hitl_workflow.compile(checkpointer=memory)

print("Human-in-the-loop workflow compiled!")

In [ ]:
# Run the HITL workflow
hitl_state = {
    "topic": "Why DGX Spark is perfect for local AI development",
    "content_type": "tweet",
    "draft": "",
    "revised_draft": "",
    "feedback": "",
    "approved": False,
    "revision_count": 0,
    "max_revisions": 3,
    "messages": []
}

# Config for the thread (enables persistence)
config = {"configurable": {"thread_id": "demo-thread-1"}}

print("\n" + "="*60)
print("RUNNING HUMAN-IN-THE-LOOP WORKFLOW")
print("="*60 + "\n")

result = hitl_app.invoke(hitl_state, config)

print(f"\nFinal revision count: {result['revision_count']}")

---

## Part 8: Interrupt and Resume

With checkpointing, we can pause workflows and resume later.

In [ ]:
# Create a workflow that can be interrupted

def wait_for_approval(state: ContentState) -> ContentState:
    """This node represents waiting for human approval."""
    print("\n⏳ WORKFLOW PAUSED - Waiting for human approval...")
    print(f"Current draft:\n{state.get('revised_draft') or state['draft']}")
    state['messages'].append("Waiting for approval")
    return state

# Build interruptible workflow
interrupt_workflow = StateGraph(ContentState)

interrupt_workflow.add_node("generate", generate_draft)
interrupt_workflow.add_node("wait_approval", wait_for_approval)
interrupt_workflow.add_node("publish", publish_content)

interrupt_workflow.set_entry_point("generate")
interrupt_workflow.add_edge("generate", "wait_approval")
interrupt_workflow.add_edge("wait_approval", "publish")
interrupt_workflow.add_edge("publish", END)

# Compile with interrupt BEFORE wait_approval
interrupt_memory = MemorySaver()
interrupt_app = interrupt_workflow.compile(
    checkpointer=interrupt_memory,
    interrupt_before=["wait_approval"]  # Interrupt before this node
)

print("Interruptible workflow compiled!")

In [ ]:
# Start the workflow (will pause before approval)
interrupt_state = {
    "topic": "Tips for running 70B models on DGX Spark",
    "content_type": "blog",
    "draft": "",
    "revised_draft": "",
    "feedback": "",
    "approved": False,
    "revision_count": 0,
    "max_revisions": 2,
    "messages": []
}

thread_config = {"configurable": {"thread_id": "interrupt-demo"}}

print("Starting workflow (will interrupt before approval)...")
print("="*60)

# This will run until the interrupt point
partial_result = interrupt_app.invoke(interrupt_state, thread_config)

print("\n" + "="*60)
print("Workflow paused! Draft is ready for review.")
print(f"Messages so far: {partial_result['messages']}")

In [ ]:
# Later: Resume the workflow from where it left off
print("\nResuming workflow...")
print("="*60)

# Resume by passing None (uses saved state)
final_result = interrupt_app.invoke(None, thread_config)

print(f"\nWorkflow complete! Messages: {final_result['messages']}")

---

## ⚠️ Common Mistakes

### Mistake 1: Forgetting to Handle All Conditional Paths

In [ ]:
# ❌ Wrong: Missing a possible return value
# def route_incomplete(state):
#     if state['approved']:
#         return "publish"
#     # What if not approved? No return! Will error.

# ✅ Right: Handle all cases
def route_complete(state: ContentState) -> str:
    if state['approved']:
        return "publish"
    else:
        return "revise"  # Always have a default!

print("Always handle all conditional branches!")

### Mistake 2: Not Using Type Hints for State

In [ ]:
# ❌ Wrong: Using plain dict (no type checking)
# def bad_node(state: dict) -> dict:
#     state['misppeled_key'] = "oops"  # No warning!
#     return state

# ✅ Right: Use TypedDict for type safety
class SafeState(TypedDict):
    correctly_spelled_key: str

def good_node(state: SafeState) -> SafeState:
    state['correctly_spelled_key'] = "safe!"  # IDE warns if wrong
    return state

print("TypedDict provides type safety and IDE support!")

### Mistake 3: Infinite Loops

In [ ]:
# ❌ Wrong: No exit condition from loop
# workflow.add_edge("revise", "review")
# workflow.add_edge("review", "revise")  # Always revises forever!

# ✅ Right: Include exit conditions
# 1. Track iteration count
# 2. Have a max_iterations limit
# 3. Include approval state that breaks the loop

print("""Always ensure loops can exit:
- Track revision count
- Set max iterations
- Have approval conditions""")

---

## 🎉 Checkpoint

You've learned:
- ✅ How LangGraph enables complex workflow orchestration
- ✅ Building stateful graphs with nodes and edges
- ✅ Conditional routing based on state
- ✅ Human-in-the-loop approval patterns
- ✅ Interrupting and resuming workflows

---

## 🚀 Challenge (Optional)

Build a more complex workflow that includes:
1. Multiple content types (choose between blog, tweet, email)
2. A "research" step before drafting
3. Parallel review by multiple reviewers
4. Final manager approval

---

## 📖 Further Reading

- [LangGraph Documentation](https://langchain-ai.github.io/langgraph/)
- [Human-in-the-Loop Guide](https://langchain-ai.github.io/langgraph/how-tos/human-in-the-loop/)
- [Persistence & Memory](https://langchain-ai.github.io/langgraph/how-tos/persistence/)

---

## 🧹 Cleanup

In [ ]:
# Comprehensive cleanup for DGX Spark
import gc

# Clear GPU memory if available
try:
    import torch
    if torch.cuda.is_available():
        torch.cuda.synchronize()
        torch.cuda.empty_cache()
        allocated = torch.cuda.memory_allocated() / 1e9
        print(f"✅ GPU memory cleared ({allocated:.2f} GB still allocated)")
except ImportError:
    pass

# Python garbage collection
gc.collect()
print("✅ Cleanup complete!")

---

## 🎓 Summary

In this notebook, you mastered LangGraph for building sophisticated agent workflows:

1. **State Management**: TypedDict for structured, type-safe state
2. **Nodes**: Functions that transform state
3. **Edges**: Define workflow flow (linear and conditional)
4. **Human-in-the-Loop**: Approval checkpoints
5. **Persistence**: Interrupt and resume with memory

**When to use LangGraph:**
- Complex workflows with branching
- Human approval required
- Need to persist state across sessions
- Multi-step processes with loops

**Next up:** Lab 3.4.5 - Multi-Agent System with CrewAI